In this notebook we'll explore topic modeling to discover broad themes in a collection of movie summaries.

In [1]:
import nltk
import re
import gensim
from gensim import corpora
import operator

nltk.download('stopwords')
from nltk.corpus import stopwords

import numpy as np
import random

random.seed(1)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/castroian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def read_stopwords(filename):
    stopwords={}
    with open(filename) as file:
        for line in file:
            stopwords[line.rstrip()]=1
    return stopwords

Since we're running topic modeling on texts with lots of names, we'll add the Jockers list of stopwords (which includes character names) to our stoplist.

In [3]:
stop_words = {k:1 for k in stopwords.words('english')}
# create dict of key:1 for all words in nltk stopwords
stop_words.update(read_stopwords("../data/jockers.stopwords"))
# add jockers words
stop_words["'s"]=1 
stop_words=list(stop_words.keys())

In [4]:
def filter(word, stopwords):
    
    """ Function to exclude words from a text """
    
    # no stopwords
    if word in stopwords:
        return False
    
    # has to contain at least one letter
    if re.search("[A-Za-z]", word) is not None:
        return True
    
    return False

In [5]:
def read_docs(plotFile, metadataFile, stopwords):
    
    names={}
    box={}
    
    with open(metadataFile, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            idd=cols[0]
            name=cols[2]
            boxoffice=cols[4]
            if len(boxoffice) != 0:
                box[idd]=int(boxoffice)
                names[idd]=name
    
    n=5000
    target_movies={}


    sorted_box = sorted(box.items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_box[:n]:
        target_movies[k]=names[k]
    
    docs=[]
    names=[]
   
    with open(plotFile, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            idd=cols[0]
            text=cols[1]
            
            if idd in target_movies:
                tokens=nltk.word_tokenize(text.lower())
                tokens=[x for x in tokens if filter(x, stopwords)]
                docs.append(tokens)
                name=target_movies[idd]
                names.append(name)
                
    return docs, names

We'll read in summaries of the 5,000 movies with the highest box office revenues.

In [6]:
metadataFile="../data/movie.metadata.tsv"
plotFile="../data/plot_summaries.txt"
data, doc_names=read_docs(plotFile, metadataFile, stop_words)

In [9]:
data[1]

['seventh-day',
 'adventist',
 'church',
 'pastor',
 'chamberlain',
 'wife',
 'sons',
 'nine-week-old',
 'daughter',
 'azaria',
 'camping',
 'holiday',
 'outback',
 'baby',
 'sleeping',
 'tent',
 'family',
 'enjoying',
 'barbecue',
 'fellow',
 'campers',
 'cry',
 'heard',
 'returns',
 'tent',
 'check',
 'azaria',
 'certain',
 'sees',
 'dingo',
 'mouth',
 'running',
 'approaches',
 'discovers',
 'infant',
 'missing',
 'joins',
 'forces',
 'success',
 'assumed',
 'saw',
 'animal',
 'carrying',
 'child',
 'subsequent',
 'inquest',
 'rules',
 'account',
 'events',
 'true',
 'tide',
 'public',
 'opinion',
 'turns',
 'chamberlains',
 'stoic',
 'cold-hearted',
 'accepting',
 'disaster',
 'befallen',
 'gossip',
 'begins',
 'swell',
 'accepted',
 'statements',
 'fact',
 'couple',
 'beliefs',
 'widely',
 'practised',
 'country',
 'media',
 'report',
 'rumour',
 'name',
 'azaria',
 'means',
 'sacrifice',
 'wilderness',
 'public',
 'quick',
 'believe',
 'decapitated',
 'baby',
 'pair',
 'scissors'

We will convert the movie summaries into a bag-of-words representation using gensim's [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html) methods.

In [13]:
# Create vocab from data; restrict vocab to only the top 10K terms that show up in at least 5 documents 
# and no more than 50% of all documents

dictionary = corpora.Dictionary(data)
dictionary.filter_extremes(no_below=5, no_above=.5, keep_n=10000)

In [14]:
# Replace dataset with numeric ids words in vocab (and exclude all other words)
corpus = [dictionary.doc2bow(text) for text in data]

In [16]:
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 2),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 2),
  (14, 1),
  (15, 2),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 2),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 2),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 2),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 2),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 4),
  (51, 2),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 2),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 9),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 2),
  (77, 1),
  (78, 1),
  (79, 2),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 4),
  (91, 1)

In [12]:
num_topics=20

Now let's run a topic model on this data using gensim's built-in LDA.

In [14]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics, 
                                           passes=10,
                                           alpha='auto')

We can get a sense of what the topics are by printing the top 10 words with highest $P(word \mid topic)$ for each topic

In [15]:
for i in range(num_topics):
    print("topic %s:\t%s" % (i, ' '.join([term for term, freq in lda_model.show_topic(i, topn=10)])))

topic 0:	house room father finds night mother begins body tells goes
topic 1:	bond plane white snow flight bomb airport penguins escape master
topic 2:	gang police men boone train group town escape killed west
topic 3:	museum kane phantom fraternity pink pierce mars baron fowler adams
topic 4:	doc dracula satan ghost hell god soul connor kill priest
topic 5:	case prison murder president dr. judge trial evidence court cia
topic 6:	police money car fbi bank train kill agent drug smith
topic 7:	mr. mrs. children find boys friends kids christmas family help
topic 8:	castle dragon sword infected silver vampire stone kill treasure magic
topic 9:	de duke jaguar beck la gibson flynn cardinal bowman pope
topic 10:	escape team dr. find group world rescue island kill killed
topic 11:	earth alien space ash planet robot aliens ship find city
topic 12:	film band show music dance stage performance concert studio club
topic 13:	tells time day night asks n't goes party house leaves
topic 14:	father fam

Another way of understanding topics is to print out the documents that have the highest topic representation -- i.e., for a given topic $k$, the documents with highest $P(topic=k | document)$.  How much do the documents listed here align with your understanding of the topics?

In [16]:
topic_model=lda_model 

topic_docs=[]
for i in range(num_topics):
    topic_docs.append({})
for doc_id in range(len(corpus)):
    doc_topics=topic_model.get_document_topics(corpus[doc_id])
    for topic_num, topic_prob in doc_topics:
        topic_docs[topic_num][doc_id]=topic_prob

for i in range(num_topics):
    print("%s\n" % ' '.join([term for term, freq in topic_model.show_topic(i, topn=10)]))
    sorted_x = sorted(topic_docs[i].items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_x[:5]:
        print("%s\t%.3f\t%s" % (i,v,doc_names[k]))
    print()
    
    

house room father finds night mother begins body tells goes

0	0.952	Paranormal Activity 4
0	0.952	The Haunting in Connecticut
0	0.927	Paranormal Activity 3
0	0.909	A Nightmare on Elm Street 2: Freddy's Revenge
0	0.908	The Blair Witch Project

bond plane white snow flight bomb airport penguins escape master

1	0.864	Batman Forever
1	0.849	Snakes on a Plane
1	0.849	Never Say Never Again
1	0.750	From Russia with Love
1	0.748	Octopussy

gang police men boone train group town escape killed west

2	0.843	Deuces Wild
2	0.802	Assault on Precinct 13
2	0.791	Good Morning, Vietnam
2	0.784	The Warriors
2	0.743	Butch Cassidy and the Sundance Kid

museum kane phantom fraternity pink pierce mars baron fowler adams

3	0.682	Night at the Museum: Battle of the Smithsonian
3	0.579	The First Great Train Robbery
3	0.470	The Kid
3	0.377	Oh, God! Book II
3	0.323	Night at the Museum

doc dracula satan ghost hell god soul connor kill priest

4	0.765	Van Helsing
4	0.760	End of Days
4	0.564	The Restless
4	0.534